In [3]:
!pip install torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 2.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.9.0 which is incompatible.


In [ ]:
!pip install spacy

In [ ]:
!pip install nltk

In [1]:
import pandas as pd

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.legacy import data

torch.manual_seed(1)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Data can be downloaded in https://www.kaggle.com/datasets/ferno2/training1600000processednoemoticoncsv

In [ ]:
data_tweet=pd.read_csv('training.1600000.processed.noemoticon.csv',engine='python', header=None )

In [ ]:
data_tweet.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data_tweet.shape

(1600000, 6)

In [ ]:
data_tweet[0].value_counts()

4    800000
0    800000
Name: 0, dtype: int64

In [ ]:
data_tweet.rename(columns={0:'label',1:'id',2:'date',3:'flag',4:'user',5:'text'}, inplace=True)

In [ ]:
data_tweet.head()

,label,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data_tweet['label']=data_tweet['label'].replace([4],1)

In [ ]:
data_tweet.sample(10000).to_csv("train-processed-sample.csv", header=None, index=None)

In [ ]:
device = "cuda"

1. prepare data
2. preprocessing: cleaning, tokenize, lematization
3. Build vocabulary/corpus
4. Vectorization

i like playing music: 'i', 'like','playing','music'
'i','like','play','music'

In [ ]:
LABEL = data.LabelField()
TWEET = data.Field(tokenize='spacy', lower=True)

In [ ]:
fields = [('label',LABEL), ('id',None),('date',None),('flag',None), ('user',None),
('text', TWEET)]

In [ ]:
twitter_data=data.TabularDataset(
    path="/content/drive/My Drive/Materi Kuliah/Deep Learning/train-processed-sample.csv",
    format="CSV",
    fields=fields,
    skip_header=False)

In [ ]:
vars(twitter_data.examples[1])

{'label': '0',
 'text': ['train',
  'delays',
  'this',
  'morning',
  ',',
  'my',
  'journey',
  'took',
  'a',
  'while',
  ',',
  'a',
  'bit',
  'annoying',
  'as',
  'it',
  'has',
  'made',
  'friday',
  'feel',
  'much',
  'much',
  'longer',
  '-',
  'boo']}

In [ ]:
(train,test,valid)=twitter_data.split(split_ratio=[0.8,0.1,0.1])

In [ ]:
vars(train.examples[1])

{'label': '1', 'text': ['everything', 'ends', 'well']}

In [ ]:
(len(train),len(test),len(valid))

(8000, 1000, 1000)

In [ ]:
vocab_size = 20000
TWEET.build_vocab(train, max_size = vocab_size)
LABEL.build_vocab(train)

In [ ]:
TWEET.vocab.freqs.most_common(10)

[('i', 5022),
 ('!', 4494),
 ('.', 3995),
 (' ', 2959),
 ('to', 2840),
 ('the', 2609),
 (',', 2443),
 ('a', 1883),
 ('my', 1637),
 ('and', 1606)]

In [ ]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train, valid, test),
                                                                           batch_size = 32,device = device)

Training: mini batch, batch, online learning

8000 data -> batch,
32, 32,32...->mini batch
1, -> online learning

In [ ]:
8000/32

250.0

In [ ]:
import torch.nn as nn
class OurFirstLSTM(nn.Module):
  def __init__(self, hidden_size, embedding_dim, vocab_size):
    super(OurFirstLSTM, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.encoder = nn.LSTM(input_size=embedding_dim,
                           hidden_size=hidden_size, num_layers=1)
    self.predictor = nn.Linear(hidden_size, 2)
  def forward(self, seq):
    output, (hidden,_) = self.encoder(self.embedding(seq)) 
    preds = self.predictor(hidden.squeeze(0))
    return preds


In [ ]:
model = OurFirstLSTM(100,300, 20002)
model.to(device)

OurFirstLSTM(
  (embedding): Embedding(20002, 300)
  (encoder): LSTM(300, 100)
  (predictor): Linear(in_features=100, out_features=2, bias=True)
)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=2e-2)
criterion = nn.CrossEntropyLoss()
def train(epochs, model, optimizer, criterion, train_iterator): 
  for epoch in range(1, epochs + 1):
    training_loss = 0.0
    valid_loss = 0.0
    model.train()
    for batch_idx, batch in enumerate(train_iterator):
      optimizer.zero_grad()
      predict = model(batch.text)
      loss = criterion(predict,batch.label)
      loss.backward()
      optimizer.step()
      training_loss += loss.data.item() * batch.text.size(0)
      training_loss /= len(train_iterator)
      print('Epoch: {}, Training Loss: {:.2f}'.format(epoch, training_loss))

      

In [ ]:
train(10, model, optimizer, criterion, train_iterator)

Epoch: 1, Training Loss: 0.09
Epoch: 1, Training Loss: 0.18
Epoch: 1, Training Loss: 0.17
Epoch: 1, Training Loss: 0.13
Epoch: 1, Training Loss: 0.09
Epoch: 1, Training Loss: 0.09
Epoch: 1, Training Loss: 0.12
Epoch: 1, Training Loss: 0.15
Epoch: 1, Training Loss: 0.11
Epoch: 1, Training Loss: 0.10
Epoch: 1, Training Loss: 0.10
Epoch: 1, Training Loss: 0.10
Epoch: 1, Training Loss: 0.10
Epoch: 1, Training Loss: 0.08
Epoch: 1, Training Loss: 0.11
Epoch: 1, Training Loss: 0.09
Epoch: 1, Training Loss: 0.10
Epoch: 1, Training Loss: 0.10
Epoch: 1, Training Loss: 0.10
Epoch: 1, Training Loss: 0.08
Epoch: 1, Training Loss: 0.10
Epoch: 1, Training Loss: 0.10
Epoch: 1, Training Loss: 0.10
Epoch: 1, Training Loss: 0.09
Epoch: 1, Training Loss: 0.09
Epoch: 1, Training Loss: 0.10
Epoch: 1, Training Loss: 0.09
Epoch: 1, Training Loss: 0.09
Epoch: 1, Training Loss: 0.10
Epoch: 1, Training Loss: 0.10
Epoch: 1, Training Loss: 0.11
Epoch: 1, Training Loss: 0.10
Epoch: 1, Training Loss: 0.10
Epoch: 1, 

Data -> training
validasi
testing

epoch 1 training loss= 0.1  validation loss=0.5



epoch 50 training loss = 0.001 validation loss = 0.3
epoch 51 training loss=0.0012 valodation loss= 0.29







epoch 100 training loss=0.001   validation loss=0.0013



testing final model dari training -> output -> testing loss=0.0017

loss= selisih dari nilai actual - nilai output dari model

accuracy tp, tn, fp,fp


target = 1(positive), output= 0.99(positive) TP
target =0 (neagtive), output=0,000001(negative) TN
target=1 (positive), output=0.00001 (negative) FN
target=0 (negative), output=0.999 (positive) FP

Accuracy = (TP+TN)/(TP+TN+FP+FN) =80%
recall = TP/(TP+FN) =79%
precision = TP/(TP+FP)=81%